## 🏥 Étude de cas : Analyse du parcours patient et prédiction des résultats médicaux

🎯 **Objectif** : Étudier les facteurs influençant les coûts de soins, la durée d’hospitalisation et les résultats médicaux (`Test Results`) afin d’identifier des tendances pertinentes dans la gestion hospitalière.

Le dataset contient des données synthétiques de patients hospitalisés, incluant des informations personnelles, cliniques, financières et administratives.


In [3]:
# 📦 Import des bibliothèques essentielles
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, f_oneway
import warnings
warnings.filterwarnings("ignore")

# 🎯 Chargement du dataset
df = pd.read_csv("healthcare_dataset.csv")

# 🧹 Nettoyage initial (si besoin)
df.columns = df.columns.str.strip()  # Suppression des espaces en trop dans les noms de colonnes

# 🕒 Conversion des dates
df["Date of Admission"] = pd.to_datetime(df["Date of Admission"])
df["Discharge Date"] = pd.to_datetime(df["Discharge Date"])

# 🧮 Création d'une variable dérivée : durée d'hospitalisation
df["Hospital Stay (Days)"] = (df["Discharge Date"] - df["Date of Admission"]).dt.days

# 🔎 Aperçu rapide
df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,Hospital Stay (Days)
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,2
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,6
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,15
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,30
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,20


## 🟨 Partie 1 – Statistiques univariées : `Billing Amount`

### 📘 Notion
Analyse descriptive d’une variable quantitative (`Billing Amount`) à l’aide d’indicateurs classiques :
- Moyenne, médiane, mode
- Variance, écart-type
- Intervalle interquartile (IQR)
- Étendue (écart entre valeur max et min)

### 🎯 Objectif dans l’étude
Le `Billing Amount` correspond au montant total facturé à un patient.  
L’objectif ici est de :
- Résumer la **distribution des coûts**,
- Identifier **la variabilité**,
- Détecter d’éventuelles **valeurs extrêmes**,
- Préparer les futures comparaisons par groupe (âge, genre, type d’admission...).



### 🧮 Formules mathématiques utilisées

$$
\bar{x} = \frac{1}{n} \sum_{i=1}^{n} x_i \quad \text{(Moyenne)}
$$

$$
s^2 = \frac{1}{n - 1} \sum_{i=1}^{n}(x_i - \bar{x})^2 \quad \text{(Variance)}
$$

$$
s = \sqrt{s^2} \quad \text{(Écart\text{-}type)}
$$

$$
\text{IQR} = Q_3 - Q_1
$$

$$
\text{Étendue} = x_{\text{max}} - x_{\text{min}}
$$


In [4]:
# 📊 Statistiques descriptives sur Billing Amount
billing_amount = df["Billing Amount"].dropna()

billing_stats = {
    "Moyenne": round(billing_amount.mean(), 2),
    "Médiane": round(billing_amount.median(), 2),
    "Mode": round(billing_amount.mode()[0], 2),
    "Variance": round(billing_amount.var(), 2),
    "Écart-type": round(billing_amount.std(), 2),
    "Minimum": round(billing_amount.min(), 2),
    "Maximum": round(billing_amount.max(), 2),
    "IQR": round(billing_amount.quantile(0.75) - billing_amount.quantile(0.25), 2)
}

billing_stats


{'Moyenne': np.float64(25539.32),
 'Médiane': np.float64(25538.07),
 'Mode': np.float64(-1316.62),
 'Variance': np.float64(201965437.04),
 'Écart-type': np.float64(14211.45),
 'Minimum': np.float64(-2008.49),
 'Maximum': np.float64(52764.28),
 'IQR': np.float64(24579.28)}

### 📊 Interprétation

- **Moyenne** ≈ 25 539 € et **médiane** ≈ 25 538 € → les deux sont presque identiques, ce qui indique une **distribution relativement symétrique** du `Billing Amount`.

- **Écart-type** ≈ 14 211 € → cela montre une **grande dispersion** des montants facturés. Les patients ne paient donc pas tous le même niveau de soin : certaines factures sont bien plus élevées que d'autres.

- **Variance** très élevée (~ 20 millions) → accentue l'idée d'une **variabilité importante**, attendue pour des données monétaires.

- **IQR** ≈ 24 579 € → les 50 % des valeurs centrales sont dispersées sur un intervalle très large, ce qui montre une **hétérogénéité significative** entre les patients.

- **Maximum** = 52 764 € et **minimum** = -2 008 € → la valeur négative est **incohérente** dans ce contexte (on ne rembourse pas au patient), ce qui suggère soit une **valeur aberrante**, soit une **erreur de saisie**. À corriger ou supprimer en Partie 5.

- **Mode** = -1 316 € → invraisemblable, probablement dû à une **anomalie ou à un regroupement de valeurs aberrantes**.

📌 **Conclusion** :
- La variable `Billing Amount` présente une **très forte dispersion** et probablement des **valeurs anormales**.
- Ces éléments justifient une **analyse complémentaire sur les valeurs extrêmes** (Partie 5) ainsi que des **comparaisons par groupe** pour identifier des causes structurelles.

---


## 🟩 Partie 2 – Corrélation entre `Age` et `Billing Amount`

### 📘 Notion
Analyse bivariée entre deux variables quantitatives : **âge du patient** et **montant facturé**.  
On utilise le **coefficient de corrélation de Pearson (r)** pour mesurer la **force et la direction** de la relation linéaire entre ces deux variables.


### 🎯 Objectif dans l’étude
Cette analyse permet de :
- Identifier si l’**âge** a un impact sur le **coût des soins**,
- Vérifier s’il existe une **tendance** (ex. : plus âgé = plus coûteux ?),
- Aider à expliquer les **facteurs prédictifs des coûts** dans les parties futures (ex. : modélisation).


### 🧮 Formule de la corrélation de Pearson

$$
r = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}
         {\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2 \cdot \sum_{i=1}^{n} (y_i - \bar{y})^2}}
$$

où :
- \( x_i \) : les âges des patients
- \( y_i \) : les montants facturés
- \( r \in [-1, 1] \) : coefficient de corrélation linéaire


In [5]:
# 📊 Corrélation entre Age et Billing Amount
correlation_df = df[["Age", "Billing Amount"]].dropna()
correlation = correlation_df.corr().loc["Age", "Billing Amount"]
correlation


np.float64(-0.0038319421186176848)

### 📊 Interprétation du résultat

- Le coefficient de corrélation \( r \approx -0.0038 \) indique une **quasi-absence de lien linéaire** entre l’âge des patients et le montant facturé (`Billing Amount`).

- Cette valeur est extrêmement proche de zéro, ce qui signifie que **l'âge ne permet pas d'expliquer la variation du coût des soins** dans ce dataset.

- Il n’y a **ni tendance croissante ni décroissante** claire : des patients jeunes peuvent être très coûteux, tout comme des patients âgés peuvent ne pas l’être du tout.

- Cela peut s’expliquer par le fait que le **coût dépend davantage de la pathologie**, de la durée d'hospitalisation, ou du type d’admission plutôt que de l’âge seul.

📌 **Conclusion** :
- Cette variable ne sera **pas utile comme prédicteur principal** du coût dans une modélisation linéaire simple.
- Elle reste néanmoins intéressante à croiser **avec d'autres variables**, comme le `Medical Condition` ou le `Type d’admission`.

---


## 🟦 Partie 3 – Test de moyenne (`Billing Amount` selon `Gender`)

### 📘 Notion
Test statistique de comparaison de deux moyennes indépendantes.  
On utilise ici le **test t de Student** pour vérifier s’il existe une **différence significative** entre le coût moyen des soins pour les hommes et les femmes.


### 🎯 Objectif dans l’étude
🧾 Déterminer si les différences de genre influencent les coûts hospitaliers.
Cela permet :
- D’identifier d’éventuelles **inégalités** ou **tendances médicales/genrées**,
- De motiver une analyse plus fine (ex. selon pathologie ou assurance),
- De justifier la prise en compte du genre dans la modélisation.


### 🧮 Formule du test t de Student (échantillons indépendants)

$$
t = \frac{\bar{x}_1 - \bar{x}_2}
         {\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$

où :
- \( \bar{x}_1, \bar{x}_2 \) : moyennes des deux groupes (hommes, femmes)
- \( s_1^2, s_2^2 \) : variances des groupes
- \( n_1, n_2 \) : effectifs des groupes

On teste :
- \( H_0 \) : pas de différence entre les moyennes
- \( H_1 \) : il existe une différence significative


In [6]:
# 🧪 Test de Student entre hommes et femmes sur Billing Amount
from scipy.stats import ttest_ind

male_billing = df[df["Gender"] == "Male"]["Billing Amount"].dropna()
female_billing = df[df["Gender"] == "Female"]["Billing Amount"].dropna()

t_stat, p_value = ttest_ind(male_billing, female_billing, equal_var=False)
t_stat, p_value


(np.float64(1.1372542156167793), np.float64(0.25543693847921056))

### 📊 Interprétation

- Le test de Student donne une **valeur p = 0.255**, supérieure au seuil classique de 0.05.
- Cela signifie que la **différence de coût moyen entre les hommes et les femmes n’est pas statistiquement significative** dans ce dataset.
- On **ne peut pas rejeter l’hypothèse nulle** \( H_0 \) d’égalité des moyennes.

📌 **Conclusion** : Le **genre ne semble pas influencer significativement le coût des soins** dans cette base. On peut donc **ne pas prioriser cette variable** comme explicative du `Billing Amount` seul.
---
---

## 🟪 Partie 4 – ANOVA : `Billing Amount` selon `Admission Type`

### 📘 Notion
L’**analyse de variance à un facteur (ANOVA)** permet de comparer les moyennes d’un **même indicateur quantitatif** (`Billing Amount`) entre **plus de deux groupes** (ici les types d’admission : `Emergency`, `Urgent`, `Elective`).



### 🎯 Objectif dans l’étude
💰 Identifier si le **type d’admission** a un **effet significatif sur le coût des soins** :
- Urgences plus coûteuses ?
- Admissions planifiées moins chères ?
→ Cela éclaire les décisions médicales, économiques, ou organisationnelles.



### 🧮 Formule de l’ANOVA à un facteur (F de Fisher)

$$
F = \frac{\text{Variance inter-groupes}}{\text{Variance intra-groupes}} 
= \frac{MS_{between}}{MS_{within}}
$$

- \( H_0 \) : toutes les moyennes sont égales
- \( H_1 \) : au moins une moyenne diffère


In [7]:
# 🧪 ANOVA Billing Amount selon Admission Type
from scipy.stats import f_oneway

# Création des groupes par type d’admission
anova_groups = [group["Billing Amount"].dropna() 
                for name, group in df.groupby("Admission Type")]

# Test d’ANOVA
anova_stat, anova_pval = f_oneway(*anova_groups)
anova_stat, anova_pval


(np.float64(0.28440843646706704), np.float64(0.7524603468639571))

### 📊 Interprétation

- La statistique de test \( F \approx 0.284 \) est faible et la **valeur p ≈ 0.752** est très supérieure au seuil classique de 0.05.
- Cela signifie qu’il **n’y a pas de différence significative entre les moyennes** de `Billing Amount` selon les trois types d’admission (`Emergency`, `Urgent`, `Elective`).
- L’hypothèse nulle \( H_0 \) d’égalité des moyennes **n’est donc pas rejetée**.

📌 **Conclusion** :
Le **type d’admission n’influence pas significativement le coût des soins** dans ce dataset. Ce critère **ne semble pas déterminant** pour expliquer les différences de facturation à lui seul.

---

## 🟥 Partie 5 – Détection des valeurs aberrantes

### 📘 Notion
Les **valeurs aberrantes** sont des observations extrêmes qui s’éloignent fortement de la majorité des données.  
Elles peuvent fausser les analyses statistiques, notamment :
- La moyenne et l’écart-type
- Les tests d’hypothèses
- La qualité des modèles prédictifs

On les détecte ici à l’aide de la **méthode de l’IQR (interquartile range)**.


### 🎯 Objectif dans l’étude
🚨 Identifier des patients avec des montants facturés **anormalement bas ou élevés**, qui pourraient :
- Représenter des **erreurs de saisie**,
- Signaler des **cas extrêmes** à analyser à part,
- **Faussent les analyses statistiques** (notamment la régression).



### 🧮 Méthode de détection par IQR

On considère une valeur comme aberrante si elle vérifie :

$$
x < Q_1 - 1.5 \times IQR \quad \text{ou} \quad x > Q_3 + 1.5 \times IQR
$$

avec :
- \( IQR = Q_3 - Q_1 \) : intervalle interquartile
- \( Q_1 \) et \( Q_3 \) : premier et troisième quartile


In [8]:
# 🧮 Calcul des bornes selon la méthode IQR
Q1 = df["Billing Amount"].quantile(0.25)
Q3 = df["Billing Amount"].quantile(0.75)
IQR = Q3 - Q1

borne_inf = Q1 - 1.5 * IQR
borne_sup = Q3 + 1.5 * IQR

# 🎯 Filtrage des valeurs aberrantes
outliers = df[(df["Billing Amount"] < borne_inf) | (df["Billing Amount"] > borne_sup)]

# Résultats clés
borne_inf, borne_sup, len(outliers)


(np.float64(-23627.701022764355), np.float64(74689.43411091428), 0)

### 📊 Interprétation

- Les bornes calculées par la méthode IQR sont :
  - **Borne inférieure** ≈ -23 627 €
  - **Borne supérieure** ≈ 74 689 €

- Le dataset ne contient **aucune valeur considérée comme aberrante** selon cette méthode (nombre d'outliers détectés : **0**).

📌 **Conclusion** :
Bien que certaines valeurs de `Billing Amount` puissent sembler extrêmes (par exemple les valeurs négatives vues précédemment), **aucune n'est statistiquement considérée comme aberrante selon l’IQR**.  
Cela peut être dû à la **large dispersion naturelle** des coûts dans les données.

🔎 Une vérification complémentaire des valeurs négatives peut cependant être pertinente à titre **qualitatif ou métier**.

---

## 🟧 Partie 6 – Gestion des valeurs manquantes

### 📘 Notion
Les **valeurs manquantes (missing values)** sont des cases vides dans un jeu de données.  
Elles peuvent fausser :
- les statistiques descriptives,
- les visualisations,
- les modèles (régressions, classifications…).

On distingue :
- Les colonnes **fortement impactées** (trop de données manquantes → à supprimer),
- Les colonnes **partiellement affectées** (à compléter ou imputer),
- Les lignes isolées (à étudier selon le contexte).


### 🎯 Objectif dans l’étude
🧹 S’assurer que notre analyse et modélisation reposent sur **des données complètes, cohérentes et fiables**.

- Éviter des erreurs dans les calculs
- Préparer un dataset propre pour la régression ou la classification


### 🧮 Taux de valeurs manquantes

Pour chaque colonne \( C \), le taux de valeurs manquantes est :

$$
\text{Taux de NA}(C) = \frac{\text{Nombre de valeurs manquantes dans } C}{\text{Nombre total d'observations}} \times 100
$$

Une colonne peut être :
- conservée si \( \text{Taux} < 10\% \),
- imputée si \( 10\% < \text{Taux} < 30\% \),
- supprimée si \( \text{Taux} > 30\% \) (selon contexte).



In [9]:
# 📊 Taux de valeurs manquantes par colonne (%)
missing_percent = df.isna().mean() * 100
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)
missing_percent


Series([], dtype: float64)

### 📊 Interprétation

- ✅ Aucun champ du dataset ne présente de valeurs manquantes.
- Toutes les colonnes sont **complètes à 100 %**, ce qui facilite grandement :
  - les analyses statistiques,
  - les visualisations,
  - et les modélisations futures.

📌 **Conclusion** :
Aucune opération d’imputation ou de suppression n’est nécessaire.  
Le jeu de données est **propre et directement exploitable** pour la suite de l’étude (notamment la régression et la classification).

---

--------------

## 🧩 Mise au point sur l’approche d’analyse

L’ordre suivi dans cette étude peut sembler inhabituel, car nous avons d’abord mené des analyses descriptives et des tests statistiques avant de vérifier la qualité du dataset (valeurs aberrantes et manquantes). Cette approche reste cohérente dans un cadre exploratoire, car les résultats des premières analyses ne montraient pas d’anomalies évidentes pouvant remettre en cause leur validité. Nous avons ensuite confirmé que le dataset ne contient ni valeurs manquantes, ni outliers statistiques, ce qui valide a posteriori la fiabilité des tests déjà effectués.

L’ensemble des six premières parties montre que la variable `Billing Amount` est fortement dispersée mais difficilement explicable par des facteurs simples comme l’âge, le genre ou le type d’admission. Ces résultats orientent notre étude vers des variables plus riches ou combinées pour la suite (conditions médicales, durée d’hospitalisation...). La base est propre, et nous disposons maintenant d’un socle solide pour entrer dans une phase de modélisation plus approfondie.


----------


--------

## 🟫 Partie 7 – Régression linéaire multiple pour `Billing Amount`

### 📘 Notion
Modélisation du **coût des soins** (`Billing Amount`) en fonction de plusieurs variables explicatives :  
- Durée d’hospitalisation (`Hospital Stay (Days)`)  
- Âge du patient (`Age`)  
- Type d’admission (`Admission Type`)  
- Genre (`Gender`)  
- Assurance (`Insurance Provider`)

### 🎯 Objectif dans l’étude
Quantifier l’influence simultanée de ces facteurs sur les coûts et obtenir un **indice de qualité de l’ajustement** (R²) et une **mesure d’erreur** (RMSE) pour juger la performance prédictive.


In [18]:
# 📦 Imports
import pandas as pd
import statsmodels.api as sm

# 🔢 Sélection des variables explicatives et de la cible
features = [
    "Hospital Stay (Days)",
    "Age",
    "Admission Type",
    "Gender",
    "Insurance Provider"
]
X_raw = df[features]
y = df["Billing Amount"]

# ⚙️ Transformation des caractéristiques avec le même preprocessor
preprocessor = pipeline.named_steps["preprocessor"]
X_transformed = preprocessor.transform(X_raw)

# 🔤 Noms des nouvelles features après encodage
numeric_features = ["Hospital Stay (Days)", "Age"]
categorical_features = ["Admission Type", "Gender", "Insurance Provider"]

cat_feature_names = (
    preprocessor
    .named_transformers_["cat"]
    .get_feature_names_out(categorical_features)
).tolist()

feature_names = numeric_features + cat_feature_names

# 🐍 Construction d'un DataFrame pour statsmodels
X_sm = pd.DataFrame(X_transformed, columns=feature_names)

# ➕ Ajout de la constante (intercept)
X_sm = sm.add_constant(X_sm)

# 🚀 Ajustement du modèle OLS
model_sm = sm.OLS(y, X_sm).fit()

# 📋 Résumé détaillé
print(model_sm.summary())


                            OLS Regression Results                            
Dep. Variable:         Billing Amount   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.7022
Date:                Thu, 15 May 2025   Prob (F-statistic):              0.708
Time:                        20:27:02   Log-Likelihood:            -6.0943e+05
No. Observations:               55500   AIC:                         1.219e+06
Df Residuals:                   55490   BIC:                         1.219e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co

### 📊 Analyse de la régression linéaire multiple

Le modèle linéaire multiple a permis de tester simultanément l’influence de la durée d’hospitalisation, de l’âge, du type d’admission, du genre et de l’assurance sur le coût des soins.  
- Le **R² test ≈ -0.001** (adj ≈ 0.000) montre que ces variables n’expliquent **aucune** variation du `Billing Amount`.  
- Les **p-values** de tous les coefficients sont **> 0.05**, confirmant l’absence d’effets significatifs.  
- Le **RMSE ≈ 14 108 €** reste proche de l’écart-type initial, signe que le modèle n’a pas amélioré la prédiction par rapport à une constante.  

**Conclusion** : ce jeu de variables linéaires n’est pas suffisant pour modéliser le coût. Passons à un modèle plus flexible pour capturer d’éventuelles relations non linéaires.

----
----

## 🟧 Partie 8 – Régression polynomiale sur `Billing Amount`

### 📘 Notion
Étendre le modèle linéaire en incluant des **termes quadratiques** pour capturer d’éventuels effets non linéaires (p. ex. coûts croissant plus vite après un certain nombre de jours).

### 🎯 Objectif
Améliorer la qualité prédictive en testant une forme polynomiale sur les variables numériques (`Hospital Stay (Days)`, `Age`), tout en gardant l’encodage des catégories pour `Admission Type`, `Gender`, `Insurance Provider`.


In [20]:
# 📦 Imports
import numpy as np
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm

# 🔢 Sélection des variables
numeric = df[["Hospital Stay (Days)", "Age"]]
categorical = df[["Admission Type", "Gender", "Insurance Provider"]]

# ⚙️ Création des termes polynomiaux (degré 2)
poly = PolynomialFeatures(degree=2, include_bias=False)
num_poly = pd.DataFrame(
    poly.fit_transform(numeric),
    columns=poly.get_feature_names_out(numeric.columns),
    index=df.index
)

# 🏷️ Encodage des variables catégorielles
cat_dummies = pd.get_dummies(categorical, drop_first=True)

# 🔗 Assemblage du design matrix
X_poly = pd.concat([num_poly, cat_dummies], axis=1)

# 🛠️ Conversion des colonnes booléennes en numériques
X_poly = X_poly.astype(float)

# ➕ Ajout de la constante (intercept)
X_poly = sm.add_constant(X_poly)

# 🎯 Cible
y = df["Billing Amount"]

# 🚀 Ajustement du modèle OLS
model_poly = sm.OLS(y, X_poly).fit()

# 📋 Résumé détaillé
print(model_poly.summary())


                            OLS Regression Results                            
Dep. Variable:         Billing Amount   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.5837
Date:                Thu, 15 May 2025   Prob (F-statistic):              0.857
Time:                        20:29:25   Log-Likelihood:            -6.0943e+05
No. Observations:               55500   AIC:                         1.219e+06
Df Residuals:                   55487   BIC:                         1.219e+06
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
co


### 📊 Analyse 

Le modèle de régression logistique a été entraîné pour prédire la modalité de `Test Results` (Normal / Abnormal / Inconclusive) à partir des mêmes variables explicatives que pour la régression :

- **Hospital Stay (Days)**  
- **Age**  
- **Admission Type**, **Gender**, **Insurance Provider**

Après entraînement et évaluation sur l’échantillon de test, on reportera :

- **Accuracy**  
- **Matrice de confusion**  
- **Scores par classe** (precision, recall, f1-score)

Ces métriques permettront de conclure si les caractéristiques hospitalières et démographiques suffisent à prédire le résultat des tests médicaux.

----


## 🟩 Partie 9 – Régression logistique sur `Test Results`

### 📘 Notion
Classification multinomiale avec la **régression logistique** afin de modéliser la probabilité d’appartenir à chaque classe de `Test Results`.

### 🎯 Objectif
Évaluer si les mêmes facteurs (durée, âge, type d’admission, genre, assurance) permettent de prédire efficacement le **résultat du test médical**.


In [22]:
# 📦 Imports
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 🔢 Préparation des X et y
features = [
    "Hospital Stay (Days)",
    "Age",
    "Admission Type",
    "Gender",
    "Insurance Provider"
]
X = df[features]

# Encodage de la cible
le = LabelEncoder()
y = le.fit_transform(df["Test Results"])  # classes : Normal, Abnormal, Inconclusive

# ⚙️ Prétraitement
numeric_features = ["Hospital Stay (Days)", "Age"]
categorical_features = ["Admission Type", "Gender", "Insurance Provider"]

preprocessor = ColumnTransformer([
    ("num", StandardScaler(), numeric_features),
    ("cat", OneHotEncoder(drop="first", handle_unknown="ignore"), categorical_features),
])

# 🔗 Pipeline complet
pipeline_log = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(multi_class="multinomial", max_iter=1000))
])

# 🧪 Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 🚀 Entraînement
pipeline_log.fit(X_train, y_train)

# 🎯 Prédiction et évaluation
y_pred = pipeline_log.predict(X_test)

print("Accuracy :", round(accuracy_score(y_test, y_pred), 3))
print("\nClassification report :\n", classification_report(y_test, y_pred, target_names=le.classes_))
print("Confusion matrix :\n", confusion_matrix(y_test, y_pred))


Accuracy : 0.339

Classification report :
               precision    recall  f1-score   support

    Abnormal       0.34      0.45      0.39      3726
Inconclusive       0.34      0.23      0.27      3671
      Normal       0.34      0.34      0.34      3703

    accuracy                           0.34     11100
   macro avg       0.34      0.34      0.33     11100
weighted avg       0.34      0.34      0.33     11100

Confusion matrix :
 [[1678  791 1257]
 [1652  842 1177]
 [1621  837 1245]]


## 🏁 Conclusion du projet

### 🔍 Problématique
Étudier les facteurs influençant :
- le **coût des soins** (`Billing Amount`),  
- la **durée d’hospitalisation** (`Hospital Stay (Days)`),  
- et les **résultats médicaux** (`Test Results`),  

à partir de données synthétiques de patients hospitalisés, afin d’identifier des tendances utiles pour la gestion hospitalière.

---

### 📊 Résumé des résultats
1. **Exploration des coûts**  
   - `Billing Amount` très dispersé (moyenne ≈ 25 500 €, écart-type ≈ 14 200 €).  
   - Aucune corrélation significative avec l’âge, le genre ou le type d’admission.  
   - Pas d’outliers IQR et pas de valeurs manquantes après vérification.

2. **Modèles de régression**  
   - **Linéraire simple et multiple** : R² ≈ 0.00, coefficients non significatifs.  
   - **Polynomiale** : toujours R² ≈ 0.00, terms quadratiques non significatifs.  

3. **Classification des résultats**  
   - Régression logistique multinomiale sur `Test Results` : accuracy ≈ 0.34, proche du hasard (33 %).  
   - Les variables administratives et démographiques ne suffisent pas à prédire le résultat médical.

---

### 🏥 Conclusion
Les facteurs étudiés (`Hospital Stay`, `Age`, `Gender`, `Admission Type`, `Insurance Provider`) ne permettent pas d’expliquer ni de prédire les coûts ni les résultats médicaux dans ce jeu de données. Pour obtenir des modèles performants, il faudra intégrer des **informations cliniques plus détaillées** (diagnostics, traitements, paramètres biologiques) et/ou tester des **algorithmes plus complexes**.
